In [60]:
import numpy as np
import pandas as pd
import os
import string
import scipy.io as spio
import pandapower as pp
from pandapower.plotting.plotly import simple_plotly
from pandapower.plotting.plotly import vlevel_plotly
import pathlib
from importlib import resources

cwd = pathlib.Path.cwd()
import plotly.express as px

np.set_printoptions(linewidth=100)
from GridDataGen.utils.io import *
from GridDataGen.utils.process_network import *
from GridDataGen.utils.config import *

# Resources
- OPF: Optimal Power Flow. **A good introduction is provided here**: https://invenia.github.io/blog/2021/06/18/opf-intro/
- PF: Power Flow. **A good introduction is provided here**: https://invenia.github.io/blog/2020/12/04/pf-intro/
- pandapower documentation: https://pandapower.readthedocs.io/en/v2.0.0/index.html




# Nomenclature
- This is not best practice, but I often use the words "demand", "load", and "consumption" interchangeably to mean the amount of electrical power required.
- I sometimes refer to electrical "buses" as "nodes"

# Loading the network

- We first load the network in pandapower.
- The list of available networks in pandapower is here: https://pandapower.readthedocs.io/en/v2.0.0/networks/power_system_test_cases.html#case-gb-reduced-network

- Here, we load the 24-bus IEEE system
- IEEE test systems, like the IEEE 24-bus system, are simplified models of power grids used to test and compare methods for PF and OPF. These systems are introduced here: https://arxiv.org/pdf/1908.02788








In [61]:
grid_name = "case24_ieee_rts" 
net = load_net_from_pp(grid_name)
network_preprocessing(net) # this adds names to network elements and assigns bus types


- We load the demand profile which is obtained from powergraph  https://github.com/PowerGraph-Datasets/PowerGraph-Node
- Make sure you load the right demand profile for the network you are considering. Powergraph has load profiles for IEEE118, IEEE24, IEEE39, UK, and the name of these networks is slightly different in pandapower.



In [62]:
demand_file_grid = "case24_ieee_rts"
demand_file = resources.files(f"GridDataGen.grids.{demand_file_grid}").joinpath(
    "hourlyDemandBusnew.mat"
)

# Overview of different pandapower `net` dataframes

- pandapower stores the network data in different dataframes. We will take a look at some of them below
- Initially, the load and generation information is the one coming from the grid case file we have loaded

## Load

- `net.load` contains all load elements in the grid. Each bus can have 0 or max 1 load. The properties of the load elements include:
    - `p_mw` is the active power demand
    - `q_mvar` is the reactive power demand
    - the column `bus` indicates the bus the loads are connected to

In [ ]:
net.load

## Generators (`gen`)

Same as above, but:
- `p_mw` is the active power generated 
- there is no reactive power info before we run OPF
- `min_p_mw`, `max_p_mw`, `min_q_mvar`, `max_q_mvar` are bounds on the active (reactive) power generated 
- `vm_pu` is the voltage in per unit


In [ ]:
net.gen

## Static generators (`sgen`)

In pandapower, static generators (`sgen`s) inject active and reactive power into the grid but don’t have a voltage control variable, meaning their output doesn’t directly regulate bus voltage like a traditional generator.

In [ ]:
net.sgen

## REF/ Slack

- The slack (aka. "ref") bus is a reference node in the system used to balance the active and reactive power in the grid
- In pandapower, the `ext_grid` element is connected to the slack bus and plays this role. it has fixed voltage and phase angle, and the system can draw or get power from it to maintain balance.

In [ ]:
net.ext_grid

## Shunts

- Shunts are other elements used to balance the active and reactive power in the grid.
- Think of them as "controllable" loads

In [ ]:
net.shunt

## Lines

Lines connect buses
- `r_ohm_per_km` and `x_ohm_per_km`: Line resistance and reactance.
- `max_i_ka`: max current capacity

In [ ]:
net.line

## Transformers 

In [ ]:
net.trafo

## poly cost

`net.polycost` stores the different terms of the generation cost function

At every `gen` or `sgen` element, we have:
$$C(p_{\text{MW}}, q_{\text{MVar}}) = cp_0 + cp_1 \cdot p_{\text{MW}} + cp_2 \cdot p_{\text{MW}}^2 + cq_0 + cq_1 \cdot q_{\text{MVar}} + cq_2 \cdot q_{\text{MVar}}^2$$



In [ ]:
net.poly_cost

## Bus types

- **PV** Buses: connected to `gen`s, `sgen`s, and sometimes also `load`s
- **PQ** Buses: connected to `load`s (or nothing) and/or `shunt`s. Can **not** be connected to generation elements like `gen`, `sgen` or `ext_grid`
- Slack/ REF Bus: connected to `ext_grid`. Can also be connected to any other element  


In [ ]:
net.bus

Note that there are PQ buses which dont have any load (nor generator element, by definition)

In [ ]:
(net.bus["type"] == PQ).sum()  # number of PQ buses

In [ ]:
net.load[net.load.type == PQ].bus.nunique()  # number of PQ buses with loads

## Viz

I keep this here but I am not sure it is useful...

In [ ]:
# Quick Visualization of the loaded Grid (Matplotlib Version)
pp.plotting.simple_plot(net)

# Loading the load scenarios

We first get the load scenarios from the scenario file, the path to which we defined earlier:

In [75]:
scenarios = load_scenario_from_file(net, demand_file) 

Note that these load scenarios show many limitations. Among others, the load fluctuations over time is the same accross all buses... -> the load variations are not independent accross buses ;)

In [ ]:
df = pd.DataFrame(scenarios[:5, :].T, columns=[f"Bus {i+1}" for i in range(5)])

# Create the line plot with a legend
fig = px.line(df, title="Time Series of the Load for the First 5 Buses")
fig.update_layout(
    xaxis_title="Time Steps",
    yaxis_title="Load",
    legend_title="Buses"
)
fig.show()


- We now replace the default active power load from the case file by the active power load of the **first load scenario** (in this notebook, we will only solve one OPF/PF problem, i.e. look at one time-snapshot of the grid). 
- Note that the **reactive power is not changed** (as it is the case in the approach used by powergraph) -> we should improve this in the future


In [77]:
#  set demand at each load element according to scenario
net.load.p_mw = scenarios[:, 0][net.load.bus]

# OPF

- OPF is an optimization problem solved to determine the least cost generation dispatch (active power generation and voltage magnitude at each PV bus) to satisfy the active and reactive power load that we specified.
- Again, the active power load is now the one from the scenario file, while the reactive power load is the default one from the case file


## Solving OPF

- `net` already contains all the network parameters (e.g. line reactance, generator max capacity, ...) and the load information. Solving OPF can thus simply be done by calling `runopp()` from pandapower.

In [ ]:
pp.runopp(net, numba=True)
net.res_gen.iloc[0]

In [ ]:
net.gen.loc[0,"in_service"] = False
pp.runopp(net, numba=True)
net.res_gen.iloc[0]

after running opf, the `net.res_*` dataframes (e.g. `net.res_gen`) are added to the `net` object:

In [ ]:
net

We add the bus indices and types to the result dataframe (we will need them later)

In [25]:
# add bus index and type to dataframe of opf results
net.res_gen["bus"] = net.gen.bus
net.res_gen["type"] = net.gen.type
net.res_load["bus"] = net.load.bus
net.res_load["type"] = net.load.type
net.res_sgen["bus"] = net.sgen.bus
net.res_sgen["type"] = net.sgen.type
net.res_shunt["bus"] = net.shunt.bus
net.res_ext_grid["bus"] = net.ext_grid.bus
net.res_bus["type"] = net.bus["type"]

Note that the load stays the same as before, it is not modified during OPF, only the generation dispatch is changed

In [26]:
# assert that the load doesnt get changed
assert net.res_load.p_mw.equals(net.load.p_mw)
assert net.res_load.q_mvar.equals(net.load.q_mvar)


## Res gen

- `res_gen` now contains the optimal active and reactive power generation, as well as the voltage mag and angle

In [ ]:
net.res_gen

we can check if the constraints on the active and reactive power are respected:

In [28]:
# check active power is less than the upper bound
assert (
    net.gen.max_p_mw - net.res_gen.p_mw > -1e-4
).all(), "upper bound on active power is violated"
# check active power is more than the lower bound
assert (
    net.res_gen.p_mw - net.gen.min_p_mw
).all(), "lower bound on active power is violated"

# same for reactive power
assert (
    net.gen.max_q_mvar - net.res_gen.q_mvar > -1e-4
).all(), "upper bound on reactive power is violated"

assert  (
    net.res_gen.q_mvar - net.gen.min_q_mvar
).all(),  "lower bound on reactive power is violated"

## Res sgen

- `res_sgen` now contains the optimal active and reactive power generation

## Checking bounds on active and reactive power

In [29]:
if len(net.sgen) != 0:
    assert (
        net.sgen.max_p_mw - net.res_sgen.p_mw > -1e-4
    ).all(), "upper bound on active power is violated"
    assert (
        net.res_sgen.p_mw - net.sgen.min_p_mw > -1e-4
    ).all(), "lower bound on active power is violated"

    assert (
        net.sgen.max_q_mvar - net.res_sgen.q_mvar > -1e-4
    ).all(), "upper bound on reactive power is violated"
    assert (
        net.res_sgen.q_mvar - net.sgen.min_q_mvar > -1e-4
    ).all(), "lower bound on reactive power is violated"

## Slack

- `ext_grid` now has active and reactive power: (positive means supplying power)

In [ ]:
net.res_ext_grid

## Checking power balance

We can check that some other basic constraints are satisfied. For example, the total power generated should be equal to the total power consumed + loss

- We sum the active and reactive power of all generator elements (gen, static gen and slack) connected to each bus:

In [ ]:
all_gens = (
    pd.concat([net.res_gen, net.res_sgen, net.res_ext_grid])[["p_mw", "q_mvar", "bus"]]
    .groupby("bus")
    .sum()
)
all_gens

- total generation across all buses:

In [ ]:
all_gens.sum()

- We can now sum the active and reactive power of all load elements (loads, shunts and transformers) + losses (line and transformer):

In [ ]:
all_consumption_p_mw = (
    net.res_load.p_mw.sum()
    + net.res_line.pl_mw.sum()
    + net.res_trafo.pl_mw.sum()
    + net.res_shunt.p_mw.sum()
)  # active
all_consumption_p_mw

all_consumption_q_mvar = (
    net.res_load.q_mvar.sum()
    + net.res_line.ql_mvar.sum()
    + net.res_trafo.ql_mvar.sum()
    + net.res_shunt.q_mvar.sum()
)  # reactive
all_consumption_q_mvar



print(f"Total active consumption: {all_consumption_p_mw} MW")
print(f"Total reactive consumption: {all_consumption_q_mvar} MVar")

- The total load + losses matches the total generation!

## res bus

Note that the net power demand is also available in the `res_bus` table.

To check that, we can compute the net power consumption at each bus (i.e. all loads - all geneation)

In [34]:
all_loads = (
    pd.concat([net.res_load, net.res_shunt])[["p_mw", "q_mvar", "bus"]]
    .groupby("bus")
    .sum()
)  # all load
net_consumption = (
    pd.concat([all_loads, -all_gens])
    .groupby("bus")
    .sum()
    .reindex_like(net.res_bus[["p_mw", "q_mvar"]])
    .fillna(0)
)  # net load = load - generation

and check that the net power consumption is equal to the data in the `res_bus` table

In [35]:
assert np.allclose(net.res_bus[["p_mw", "q_mvar"]], net_consumption)

# Power flow

After solving OPF, we solve PF to get a "full snapshot" of our grid.
When solving PF, the following variables are fixed and we solve for the rest:

| **Bus Type**    | **Fixed Variables**                    | **Solved Variables**                    |
|------------------|----------------------------------------|------------------------------------------|
| **Slack (REF)** | `vm_pu` (voltage magnitude), `va_degree` (voltage angle) | `p_mw` (active power), `q_mvar` (reactive power) |
| **PV** | `p_mw` (active power), `vm_pu` (voltage magnitude) | `q_mvar` (reactive power), `va_degree` (voltage angle) |
| **PQ**    | `p_mw` (active power), `q_mvar` (reactive power) | `vm_pu` (voltage magnitude), `va_degree` (voltage angle) |



## Setting the active power and voltage magnitude of generators based on results of OPF 

For pandapower to run PF on the OPF solutions, we need to copy OPF results of OPF from the `res_gen` and `res_sgen` tables to the `gen` and `sgen` tables.
- Among the fixed variables (see table above), only `p_mw` and `vm_pu` at PV nodes got changed after running OPF and thus require us to copy their new values. Note that the `p_mw` and `q_mvar` at PQ nodes did not get changed during OPF (they are in fact the input of OPF), and that the slack voltage magnitude and angle at slack nodes are fixed parameters that can not change.
- Thus, we only set the real power `p_mw` and voltage `vm_pu` at `gen` and `sgen` elements



In [36]:
net.gen[["p_mw", "vm_pu"]] = net.res_gen[["p_mw", "vm_pu"]]

In [37]:
net.sgen["p_mw"] = net.res_sgen["p_mw"] # note that there is no voltage info at sgens, since they do not participate in the voltage control

- Note that there is no need to set the `shunt`s' power or the `ext_grid`'s power, as they are computed during the power flow (PF) calculation.
- Their purpose is precisely to correct slight mismatches in power generation and demand by generating or consuming some energy.


## <p style="color:red"> TO DO: Adding small perturbations to the OPF results </p>




Note that this is where we could add small "perturbations" to the `p_mw` and `vm_pu` of `gen`s and `sgen`s obtained from the OPF solutions we obtained, to generate many "suboptimal" generation dispatch from our single OPF solution.

For now, we contiune with the OPF solution.



## Run PF

- We run powerflow using the `runpp` function from pandapower
- This overwrites the res_tables with the PF results.

In [ ]:
pp.runpp(net)

we can access the PF solutions using the `net.res_*` tables